In [20]:
from Dataloaders import dataloader_image_20

dataset = dataloader_image_20.Dataset_imagenet_20(device='home')
tr_sampler, t_sampler, v_sampler = dataset.return_sampler()
tr_dict, t_dict, v_dict = dataset.return_dict()
class_index = dataset.return_class_index()

print(list(t_dict.values())[0][0][-18:])
print(sorted(t_dict.keys()))
print(tr_sampler)



n02099712_129.JPEG
['10010', '10025', '10027', '1003', '10033', '10035', '10038', '10045', '10050', '10055', '10059', '10064', '10068', '10076', '10078', '10079', '10082', '10087', '10088', '10089', '10091', '10096', '10099', '101', '10109', '10122', '10123', '10127', '10128', '10136', '10138', '10142', '10156', '10158', '10164', '10167', '10169', '1017', '10178', '10180', '10182', '10185', '10188', '10190', '10194', '10203', '10208', '10214', '10216', '10219', '10225', '10238', '10239', '10240', '10251', '1026', '10263', '10265', '10269', '10274', '10275', '1028', '10284', '10295', '10296', '10297', '10301', '10303', '10305', '10307', '10312', '10314', '10317', '1032', '10321', '10322', '10333', '10335', '10337', '10340', '10344', '10345', '10346', '10347', '10350', '10353', '10359', '1036', '10368', '10372', '1039', '10392', '10398', '104', '10406', '10427', '10429', '10434', '10438', '10439', '10441', '10444', '10445', '10447', '10448', '10451', '10453', '10464', '1047', '10473', '1

In [1]:
from Dataloaders import dataloader_ccpd
from Models import mobilenetv2, resnet, gatedmodel

train, test, val = dataloader_ccpd.Dataloader_ccpd_integrated(device='home')

# dataset = 'ccpd', model = mobile, CELoss vs middles
device = 'cuda:0'
# client, server = mobilenetv2.mobilenetv2_splitter(num_classes=34,
#                                         weight_root='./Weights/ccpd-small/')
client, server = resnet.resnet_splitter(weight_root='./Weights/ccpd-small/', layers = 50, num_classes=34)

# _, test, _ = dataloader_ccpd.Dataloader_ccpd_integrated()

middle_models = []
middle_size_mobile = [1,2,4,8,16,32]
import torch
for i in range (len(middle_size_mobile)):
    middle =resnet.resnet_middle(middle = middle_size_mobile[i],)
    middle.load_state_dict(torch.load('resnet_ccpd_small_middle_%s.pth'%str(middle_size_mobile[i])))
    middle_models.append(middle)

gate_models = []
for i in range (len(middle_size_mobile)):
    gate = gatedmodel.ExitGate(in_planes=middle_size_mobile[i], height=56, width=56)
    gate.load_state_dict(torch.load('resnet_ccpd_small_gate_%s.pth'%str(middle_size_mobile[i])))
    gate_models.append(gate)

# eval and cuda
client = client.eval().to(device)
server = server.eval().to(device)
for middle in middle_models:
    middle = middle.eval().to(device)
for gate in gate_models:
    gate = gate.eval().to(device)

# quantization
import torch.quantization
client = torch.quantization.quantize_dynamic(client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
# server = torch.quantization.quantize_dynamic(server, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
for middle in middle_models:
    middle = torch.quantization.quantize_dynamic(middle, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
for gate in gate_models:
    gate = torch.quantization.quantize_dynamic(gate, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)

import torch
import numpy as np
import sys
from tqdm import tqdm
from Utils import utils
with torch.no_grad():
    # get the conf for each middles
    middles_conf = np.zeros((len(middle_models), len(test.dataset)))
    for i in range(len(middle_models)):
        count = 0
        for j, (data, target) in tqdm(enumerate(test)):
            data, target = data.to(device), target.to(device)
            out = client(data).detach()
            out = middle_models[i](out)
            out = server(out)
            out = torch.nn.functional.softmax(out, dim=1)
            out = out.gather(1, target.view(-1, 1)).view(-1)
            middles_conf[i][count:count+out.shape[0]] = out.cpu().numpy()
            count += out.shape[0]
    # get the conf for the server
# plot the seaborn
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

middles_conf = pd.DataFrame(middles_conf)
middles_conf = middles_conf.T
middles_conf.columns = [str(x) for x in middle_size_mobile]
middles_conf = middles_conf.melt(var_name='middle', value_name='conf')
middles_conf['middle'] = middles_conf['middle'].apply(lambda x: int(x))

plt.figure(figsize=(10, 6))
sns.boxplot(x='middle', y='conf', data=middles_conf, width=0.5)
plt.title('MobileNet CCPD Middles Confidence')
plt.xlabel('The number of transmitted layers')
plt.xticks(np.arange(len(middle_size_mobile)), [str(x) for x in middle_size_mobile.values()])
plt.ylabel('Confidence at the target label')
plt.show()

plt.figure(figsize=(10, 6))
sns.lineplot(x='middle', y='conf', data=middles_conf)
plt.title('MobileNet CCPD Middles Confidence')
plt.xlabel('The number of transmitted layers')
plt.xticks(np.arange(len(middle_size_mobile)), [str(x) for x in middle_size_mobile.values()])
plt.ylabel('Confidence at the target label')
plt.show()


SyntaxError: invalid syntax (2006316809.py, line 49)